In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report,accuracy_score
import pandas as pd
from sklearn.datasets import load_iris

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

Using TensorFlow backend.


In [3]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [4]:
import os
from keras.preprocessing import image

global base_dir, image_paths, target_size
base_dir = './dataset'
images_dir = os.path.join(base_dir,'image')
label_filename = 'label.csv'

image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
target_size = None
# Opening the label file
label_file = open(os.path.join(base_dir, label_filename), 'r')
# Reading the lines
label_lines = label_file.readlines()
#print(image_paths)
tumor_labels = {line.split(',')[0] : line.split(',')[1] for line in label_lines[1:]}
for k, v in tumor_labels.items():
    tumor_labels[k] = v.replace('\n', '')
#tumor_labels[:,1] = {line.replace('\n', '') for line in tumor_labels[1:]}
for k in tumor_labels:
    if 'no_tumor' in tumor_labels[k]:
        tumor_labels[k] = [0, 0, 0, 1]
    elif 'meningioma_tumor' in tumor_labels[k]:
        tumor_labels[k] = [1, 0, 0, 0]
    elif 'glioma_tumor' in tumor_labels[k]:
        tumor_labels[k] = [0, 1, 0, 0]
    elif 'pituitary_tumor' in tumor_labels[k]:
        tumor_labels[k] = [0, 0, 1, 0]
    else:
        tumor_labels[k] = [0, 0, 0, 0]
    #tumor_labels[k].contains() == 
#print(tumor_labels)

if os.path.isdir(images_dir):
    all_features = []
    #         all_features = []
    all_labels = []
    for img_path in image_paths:
         #No need for this 
            # file_name= img_path.split('/')[-1]
        #file_name= img_path.split('.')[1].split('/')[-1] + '.jpg'
        file_name= img_path.split('\\')[-1]
        # load image
        img = image.img_to_array(image.load_img(img_path, color_mode='grayscale'))
        #img = image.img_to_array(image.load_img(img_path, color_mode='grayscale', target_size=target_size, interpolation='bicubic'))
        img = np.reshape(img, (512,512, 1))
        # Getting rid of rgb data to greyscale
        #img = np.squeeze(img)
        if img is not None:
            all_features.append(img)
            all_labels.append(tumor_labels[file_name])


In [5]:
print(np.array(all_labels))

[[1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]]


In [6]:
x = np.array(all_features)
y = np.array(all_labels)
# print(xtrain.shape)
#x = np.squeeze(x)
#print(xtrain.shape)
x /= 255
#x = np.reshape(x, (x.shape[0], (512*512)))
xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size=0.80, random_state=10) 

#img = mpimg.imread('your_image.png')
#imgplot = plt.imshow(all_features[1], cmap='Greys_r')



In [7]:
x

array([[[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]],


       [[[0.13725491],
         [0.14117648],
         [0.14

In [5]:
ytrain

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0]])

In [6]:
print(xtrain.shape)
print(ytrain.shape)

(2400, 512, 512, 1)
(2400, 4)


In [5]:
def cnnmodel():
    cnn_model = Sequential()
    cnn_model.add(Dense(8, input_dim=4, activation='relu'))
    cnn_model.add(Dense(3, activation='softmax'))
    cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return  cnn_model

In [8]:
cnnmodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


In [6]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(512,512,1)))
#cnn_model.add(Conv2D(64, (3,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(64, (3,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(128, (3,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.25))
cnn_model.add(Dense(4, activation='softmax'))

cnn_model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [7]:
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 510, 510, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 255, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 128)    

In [8]:
final_model = cnn_model.fit(xtrain, ytrain, batch_size=10, epochs=10, validation_data=(xtest, ytest))


Train on 2400 samples, validate on 600 samples
Epoch 1/10
2400/2400 [==============================] - 35s 15ms/step - loss: 1.1615 - accuracy: 0.5067 - val_loss: 0.8093 - val_accuracy: 0.6183
Epoch 2/10
2400/2400 [==============================] - 22s 9ms/step - loss: 0.7531 - accuracy: 0.6717 - val_loss: 0.6577 - val_accuracy: 0.7400
Epoch 3/10
2400/2400 [==============================] - 22s 9ms/step - loss: 0.5645 - accuracy: 0.7729 - val_loss: 0.7801 - val_accuracy: 0.6850
Epoch 4/10
2400/2400 [==============================] - 22s 9ms/step - loss: 0.4251 - accuracy: 0.8296 - val_loss: 0.5677 - val_accuracy: 0.7867
Epoch 5/10
2400/2400 [==============================] - 22s 9ms/step - loss: 0.3302 - accuracy: 0.8758 - val_loss: 0.5049 - val_accuracy: 0.8283
Epoch 6/10
2400/2400 [==============================] - 22s 9ms/step - loss: 0.2573 - accuracy: 0.9000 - val_loss: 0.4964 - val_accuracy: 0.8333
Epoch 7/10
2400/2400 [==============================] - 22s 9ms/step - loss: 0.208

In [ ]:
#classification_report(ytest, )

In [13]:
# final_model = KerasClassifier(build_fn=cnn_model, epochs=200, batch_size=5, verbose=0)

In [16]:
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(final_model, xtrain, ytrain, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_24 (Dense)             (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 209, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 151, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 1154, in fit
    batch_size=batch_size)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 579, in _

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 209, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 151, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 1154, in fit
    batch_size=batch_size)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 579, in _

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_28 (Dense)             (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


...

C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 209, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 151, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 1154, in fit
    batch_size=batch_size)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 579, in _

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_40 (Dense)             (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 209, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 151, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 1154, in fit
    batch_size=batch_size)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 579, in _

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_42 (Dense)             (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________
Baseline: nan% (nan%)


C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 209, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\wrappers\scikit_learn.py", line 151, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 1154, in fit
    batch_size=batch_size)
  File "C:\Users\serka\anaconda3\envs\lab\lib\site-packages\keras\engine\training.py", line 579, in _